In [102]:
import pandas as pd
import numpy as np
import os
import gc
import time
import re
import ast
import country_converter as coco
import time
import requests
import json
from urllib.parse import quote_plus
from collections import Counter
from functools import lru_cache

# Manage API Keys
from dotenv import load_dotenv
load_dotenv()

import logging
coco_logger = logging.getLogger('country_converter.country_converter')
coco_logger.setLevel(logging.ERROR)

In [2]:
DF_input = input().strip()

  C:\Users\svalb\OneDrive\Escritorio\Data_40_years_cancer_studies\


In [3]:
entities_database_input = input().strip()

 C:\Users\svalb\OneDrive\Escritorio\Data_40_years_cancer_studies\resources\allCountries_clean.csv


In [52]:
institutions_database_input = input().strip()

 C:\Users\svalb\OneDrive\Escritorio\Data_40_years_cancer_studies\resources\ScimagoIR 2025 - Overall Rank.csv


In [112]:
df=pd.read_csv(DF_input+"parsedX_100000.csv")

In [5]:
entities_database = pd.read_csv(entities_database_input)

In [54]:
institutions_database = pd.read_csv(institutions_database_input, sep=";")

In [55]:
institutions_database

,Global Rank,Institution,Country,Sector,Best Country Quartile
0,1,Chinese Academy of Sciences *,CHN,Government,1
1,2,Ministry of Education of the People's Republic...,CHN,Government,1
2,3,Centre National de la Recherche Scientifique *,FRA,Government,1
3,4,Harvard University *,USA,Universities,1
4,5,University of Chinese Academy of Sciences *,CHN,Universities,1
...,...,...,...,...,...
9751,9752,Iranian Research Institute of Plant Protection,IRN,Government,4
9752,9753,Al-Turath University College,IRQ,Universities,4
9753,9754,Ecole Normale Superieure de Kouba Bachir el Ib...,DZA,Universities,4
9754,9755,Semashko National Research Institute of Public...,RUS,Health,4


In [57]:
# Remove " *" from institutions when present
institutions_database["Institution"] = institutions_database["Institution"].apply(lambda x: x[:-2] if x[-2:] == " *" else x)

In [6]:
entities_database.head()

,name,asciiname,alternatenames,country code
0,Xixerella,Xixerella,NaN,AD
1,Vila,Vila,"Casas Vila,Vila",AD
2,Tossalet i Vinyals1,Tossalet i Vinyals1,"Tossalet i Vin'jal's,Tossalet i Vinyals,Тоссал...",AD
3,Sornàs,Sornas,"Sornas,Sornàs",AD
4,Soldeu,Soldeu,"Sol'deu,Soldeu,surudeu,swldw,Сольдеу,סולדאו,سو...",AD


In [105]:
institutions_database.head()

,Global Rank,Institution,Country,Sector,Best Country Quartile
0,1,Chinese Academy of Sciences,CHN,Government,1
1,2,Ministry of Education of the People's Republic...,CHN,Government,1
2,3,Centre National de la Recherche Scientifique,FRA,Government,1
3,4,Harvard University,USA,Universities,1
4,5,University of Chinese Academy of Sciences,CHN,Universities,1


In [115]:
df.loc[df["PMID"]==1279132]["Authors"].values

array(["[{'Name': 'G Karp', 'Affiliation': ['Department of Medicine, UMDNJ-Robert Wood Johnson Medical School.']}, {'Name': 'K Nahum', 'Affiliation': []}]"],
      dtype=object)

In [ ]:
# Instance conerter
cc = coco.CountryConverter()

In [108]:
# Define a dictionary for US states. 
# Many article's affiliation have the state name/ code and not "United States"
us_states = [
    "Alabama", "AL", "Al",
    "Alaska", "AK", "Ak",
    "Arizona", "AZ", "Az",
    "Arkansas", "AR", "Ar",
    "California", "CA", "Ca",
    "Colorado", "CO", "Co",
    "Connecticut", "CT", "Ct",
    "Delaware", "DE", "De",
    "Florida", "FL", "Fl",
    "Georgia", "GA", "Ga",
    "Hawaii", "HI", "Hi",
    "Idaho", "ID", "Id",
    "Illinois", "IL", "Il",
    "Indiana", "IN", "In",
    "Iowa", "IA", "Ia",
    "Kansas", "KS", "Ks",
    "Kentucky", "KY", "Ky",
    "Louisiana", "LA", "La",
    "Maine", "ME", "Me",
    "Maryland", "MD", "Md",
    "Massachusetts", "MA", "Ma",
    "Michigan", "MI", "Mi",
    "Minnesota", "MN", "Mn",
    "Mississippi", "MS", "Ms",
    "Missouri", "MO", "Mo",
    "Montana", "MT", "Mt",
    "Nebraska", "NE", "Ne",
    "Nevada", "NV", "Nv",
    "New Hampshire", "NH", "Nh",
    "New Jersey", "NJ", "Nj",
    "New Mexico", "NM", "Nm",
    "New York", "NY", "Ny",
    "North Carolina", "NC", "Nc",
    "North Dakota", "ND", "Nd",
    "Ohio", "OH", "Oh",
    "Oklahoma", "OK", "Ok",
    "Oregon", "OR", "Or",
    "Pennsylvania", "PA", "Pa",
    "Rhode Island", "RI", "Ri",
    "South Carolina", "SC", "Sc",
    "South Dakota", "SD", "Sd",
    "Tennessee", "TN", "Tn",
    "Texas", "TX", "Tx",
    "Utah", "UT", "Ut",
    "Vermont", "VT", "Vt",
    "Virginia", "VA", "Va",
    "Washington", "WA", "Wa",
    "West Virginia", "WV", "Wv",
    "Wisconsin", "WI", "Wi",
    "Wyoming", "WY", "Wy",
    "District of Columbia", "DC", "Dc"
]

In [124]:
# Precompute lookup tables
institution_to_country = dict(zip(institutions_database["Institution"], institutions_database["Country"]))
entity_to_country = dict(zip(entities_database["name"], entities_database["country code"]))
all_institutions_set = set(institution_to_country.keys())

# Compiled regex list if regex matching is required
compiled_institution_patterns = [(inst, re.compile(re.escape(inst))) for inst in institution_to_country]

@lru_cache(maxsize=1000)
def get_country_from_text(text):
    """Fast country name conversion with cache."""
    result = cc.convert(names=text, to='name_short')
    return result if result != "not found" else None

def extract_country_with_states(cell_content):
    if pd.isna(cell_content):
        return None, None

    try:
        parsed_content = ast.literal_eval(cell_content)
    except:
        return None, None

    if not isinstance(parsed_content, list):
        return None, None

    countries = []
    text_blob = []

    for el in parsed_content:
        if isinstance(el, dict):
            text = el.get("text", "").strip(".") # Remove points as trailing spaces afterbefore texts
            label = el.get("label", "")

            text_blob.append(text)

            if text in us_states:
                return "United States"

            if label in {"LOC", "GPE"}:
                country = get_country_from_text(text)
                if country:
                    countries.append(country)

    if len(countries) == 1:
        return countries[0], "Direct_country"
    elif len(countries) > 1:
        # Return most common
        return Counter(countries).most_common(1)[0][0], "Most_common_list_countries"

    # No matches from LOC/GPE, try ORG matching
    entity_text = " ".join(text_blob)
    for el in reversed(parsed_content):
        if el.get("label") == "ORG":
            name = el["text"].strip(".")

            # Direct match
            if name in institution_to_country:
                return get_country_from_text(institution_to_country[name]), "Direct_institution"

            # Fallback regex search
            matches = [inst for inst, pattern in compiled_institution_patterns if pattern.search(name)]
            if len(matches) == 1:
                return get_country_from_text(institution_to_country[matches[0]]), "Regex_institution"

    # Try entity database
    for el in reversed(parsed_content):
        name = el.get("text", "").strip(".")
        if name in entity_to_country:
            country_code = entity_to_country[name]
            country = get_country_from_text(country_code)
            if country and re.search(r'\b' + re.escape(country) + r'\b', entity_text):
                return country, "Entity_database"

    return None, None

start = time.time()
df_3[["Country", "Country_source"]] = df_3["NER_lastAuthor"].apply(extract_country_with_states).apply(pd.Series)


print(time.time()-start)

119.49887704849243


In [117]:
len(df_2[(df_2["Country"].isna())&(df_2["NER_lastAuthor"].notna())])

312

In [113]:
df_2 = df.copy(deep=True)

In [114]:
df_2 = df_2.iloc[0:1000]

In [118]:
df_2[(df_2["Country"].isna())&(df_2["NER_lastAuthor"].notna())].head()

,Type,PMID,DOI,Journal,Title,Abstract,Authors,MeshHeadings,Chemicals,PublicationTypes,PublicationDate,Language,Keywords,NER_lastAuthor,Country
2,Article,1279132,10.1007/BF00172767,Journal of neuro-oncology,Hyperventilation as the initial manifestation ...,We present the case of a 57 year old man who d...,"[{'Name': 'G Karp', 'Affiliation': ['Departmen...",Antineoplastic Combined Chemotherapy Protocols...,"['Vincristine', 'Doxorubicin', 'Cyclophosphami...","['Case Reports', 'Journal Article']",1992.0,eng,NaN,"[{'text': 'Department of Medicine', 'label': '...",None
6,Article,1279163,10.1111/j.1600-0714.1992.tb01030.x,Journal of oral pathology & medicine : officia...,Reactivity of monoclonal antibodies 17.13 and ...,We studied the reactivity of monoclonal antibo...,"[{'Name': 'C A Migliorati', 'Affiliation': ['D...","Antibodies, Monoclonal, Carcinoma, Squamous Ce...","['Antibodies, Monoclonal']",['Journal Article'],1992.0,eng,NaN,[{'text': 'Department of Oral Diagnostic Scien...,None
7,Article,1279165,NaN,Journal de radiologie,[Primary mediastinal choriocarcinoma. Apropos ...,We report three cases about the diagnostic ima...,"[{'Name': 'J M Girault', 'Affiliation': [""Serv...","Adult, Antineoplastic Combined Chemotherapy Pr...","['Bleomycin', 'Etoposide', 'Cisplatin']","['Case Reports', 'Journal Article']",1992.0,fre,NaN,"[{'text': 'Service de Radiologie', 'label': 'O...",None
8,Article,1279183,10.1093/jnci/84.21.1633,Journal of the National Cancer Institute,Role of the SIKVAV site of laminin in promotio...,Angiogenesis (vascularization) has a critical ...,"[{'Name': 'M C Kibbey', 'Affiliation': ['Labor...","Amino Acid Sequence, Animals, Binding Sites, C...","['Drug Combinations', 'Laminin', 'Oligopeptide...",['Journal Article'],1992.0,eng,NaN,[{'text': 'Laboratory of Developmental Biology...,None
14,Article,1279192,10.1007/BF02887790,Journal of Tongji Medical University = Tong ji...,Studies on monoclonal anti-isotypic and anti-i...,In this study the technique of labelling the c...,"[{'Name': 'X L Wang', 'Affiliation': ['Departm...","Antibodies, Anti-Idiotypic, Antibodies, Monocl...","['Antibodies, Anti-Idiotypic', 'Antibodies, Mo...","['Journal Article', ""Research Support, Non-U.S...",1992.0,eng,NaN,"[{'text': 'Department of Immunology', 'label':...",None


In [123]:
string_parts = [el["text"] for el in ast.literal_eval(df_2.iloc[2]["NER_lastAuthor"]) if isinstance(el, dict) and "text" in el]
string = ", ".join(string_parts) + ", " if string_parts else ""
string = string[:-2]
string

'Department of Medicine, UMDNJ-Robert Wood Johnson Medical School'